In [8]:
## import neccesary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
import pickle

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load the dataset
df = pd.read_csv("D:\Bank_Churn_ANN\churn_data.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [18]:
## check the structure
df.shape

(10000, 14)

In [19]:
## check for column datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


### Data Preprocessing

In [20]:
## Data Preprocessing
## Drop irrelevent columns
df.drop(['RowNumber','CustomerId','Surname'], axis=1, inplace=True)
df.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [21]:
## check the unique values in geography and gender columns
print(df["Geography"].value_counts())

print('-------------------------------')

print(df["Gender"].value_counts())


Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64
-------------------------------
Gender
Male      5457
Female    4543
Name: count, dtype: int64


In [22]:
## encode the Gender column using label encoder
gen_label_encoder=LabelEncoder()
df["Gender"] = gen_label_encoder.fit_transform(df["Gender"])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [23]:
## Encode the Geography column using OHE
onehot_encoder = OneHotEncoder()
geo_encoder=onehot_encoder.fit_transform(df[["Geography"]])
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [24]:
onehot_encoder.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [25]:
geo_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder.get_feature_names_out(["Geography"]))

In [26]:
## combine both the dataframes
data = pd.concat([df.drop(["Geography"],axis=1),geo_df],axis=1)


In [27]:
## preprocessed data
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
## define features and target 
X=data.drop(['Exited'], axis =1)
y=data['Exited']

# Check the class distribution
y.value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [ ]:
## balanced the dataset
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Check the new classes
y_resampled.value_counts()

d:\Bank_Churn_ANN\venv\Lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
d:\Bank_Churn_ANN\venv\Lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


Exited
1    7963
0    7963
Name: count, dtype: int64

In [63]:
#split the data
X_train,X_test,y_train,y_test = train_test_split(X_resampled, y_resampled, test_size=0.2,random_state=42)

## standardize the columns
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [29]:
## save all the pickle files

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(gen_label_encoder, file)

with open('onehot_encoder.pkl', 'wb') as file:
    pickle.dump(onehot_encoder, file)

### ANN Model Building

In [64]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime 

In [65]:
## build ANN model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),        ## hidden layer with 64 neurons
    Dense(32,activation='relu'),                                        ## hidder layer with 32 neurons
    Dense(1,activation='sigmoid')                                       ## output layer
]

)

In [66]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [67]:
## define optimizer and loss function
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [68]:
## compile the model
model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['accuracy'])

In [69]:
## set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir = "log/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callbacks = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [70]:
## set up early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [71]:
## train the model

history=model.fit(
    X_train,y_train, validation_data=(X_test,y_test), epochs=100,
    callbacks=[tensorflow_callbacks,early_stopping_callback]
)

Epoch 1/100
399/399 [==============================] - 3s 5ms/step - loss: 0.4134 - accuracy: 0.8097 - val_loss: 0.3755 - val_accuracy: 0.8274
Epoch 2/100
399/399 [==============================] - 1s 3ms/step - loss: 0.3788 - accuracy: 0.8305 - val_loss: 0.3599 - val_accuracy: 0.8409
Epoch 3/100
399/399 [==============================] - 1s 3ms/step - loss: 0.3662 - accuracy: 0.8349 - val_loss: 0.3518 - val_accuracy: 0.8415
Epoch 4/100
399/399 [==============================] - 1s 4ms/step - loss: 0.3554 - accuracy: 0.8418 - val_loss: 0.3541 - val_accuracy: 0.8402
Epoch 5/100
399/399 [==============================] - 1s 4ms/step - loss: 0.3488 - accuracy: 0.8443 - val_loss: 0.3475 - val_accuracy: 0.8468
Epoch 6/100
399/399 [==============================] - 1s 4ms/step - loss: 0.3449 - accuracy: 0.8500 - val_loss: 0.3432 - val_accuracy: 0.8475
Epoch 7/100
399/399 [==============================] - 1s 4ms/step - loss: 0.3414 - accuracy: 0.8472 - val_loss: 0.3601 - val_accuracy: 0.8462

In [72]:
## save the model
model.save('model.h5')

d:\Bank_Churn_ANN\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [45]:
## load tensorboard extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir log/fit/

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

### Predictions

In [6]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import load_model
import pickle

In [7]:
### Load the trained model
model=load_model('model.h5')

## load the encoder and scaler
with open('onehot_encoder.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('label_encoder.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [8]:
# Example input data
input_data = {
    'CreditScore': 631,
    'Geography': 'France',
    'Gender': 'Female',
    'Age': 38,
    'Tenure': 2,
    'Balance': 200000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 0,
    'EstimatedSalary': 50000
}

In [9]:
# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

## convert input data into dataframe
input_df=pd.DataFrame([input_data])
input_df

## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

d:\Bank_Churn_ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


array([[-0.2142755 , -1.09499335, -0.08535128, -1.04241787,  1.98372734,
         0.80843615,  0.64920267, -1.02583358, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [10]:
## make prediction
prediction=model.predict(input_scaled)
prediction

prediction_proba = prediction[0][0]
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

1/1 [==============================] - 0s 106ms/step
The customer is likely to churn.
